In [1]:
import numpy as np
import pandas as pd
from datetime import timedelta, datetime
import os

In [2]:
# Show all columns when displaying datasets in notebook
from IPython.display import display
pd.options.display.max_columns = None

## Read and merge source files

Here we first read the source files and merge them.

In [3]:
# Areg/Wdxr/Bdsr will not be used for the following merging steps
path = "noda_cleaned"
os.listdir(path)

['Ada.csv',
 'Areg.csv',
 'Bdsr.csv',
 'Bond.csv',
 'Caze.csv',
 'Cgcd.csv',
 'Chrg.csv',
 'Dfdn.csv',
 'Disp.csv',
 'Dsum.csv',
 'Event.csv',
 'Judge.csv',
 'Parm.csv',
 'Prcn.csv',
 'Sent.csv',
 'Wdxr.csv']

In [4]:
csv_ls = os.listdir(path)
dfnames = [csv.split('.')[0] for csv in csv_ls]
df_dict = dict.fromkeys(dfnames, None)

for csv in csv_ls:
    df = pd.read_csv(os.path.join(path, csv), index_col=False)
    df_name = csv.split('.')[0]
    df_dict[df_name] = df

C:\Users\sujeong\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (5,7,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\sujeong\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\sujeong\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\sujeong\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (9,14,24) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, ce

### Preprocess selected tables

In [5]:
# Choose the latest bond for each charge
# AREG_SEQ_NBR was 1 for all records, meaning there were only 1 arrest for each 'sys_nbr - bofi_nbr' combination
idx = df_dict['Bond'].groupby(['SYS_NBR', 'BOFI_NBR', 'CHRG_SEQ_NBR'])['BOND_SEQ_NBR'].transform(max) == df_dict['Bond']['BOND_SEQ_NBR']
df_dict['Bond'] = df_dict['Bond'][idx]
df_dict['Bond']

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,BOND_SEQ_NBR,BOFI_NBR,SUB_CHARGE_CODE,BOND_STATUS_CODE,BOND_TYPE_CODE
0,84000020,1,1,1,1,255544,NaN,JD,RRJ
1,84000133,1,1,1,1,257683,NaN,RM,CMM
2,84000405,1,1,1,1,197485,NaN,RM,CMM
3,84000429,1,1,1,1,142503,DFM,NaN,CMM
4,84000603,1,1,1,1,258357,NaN,JD,RRJ
...,...,...,...,...,...,...,...,...,...
49662,99303723,1,1,3,1,310721,NaN,NaN,PSB
49663,99300543,1,1,2,1,388334,NaN,NaN,CMM
49664,99296442,2,1,1,1,369535,NaN,NaN,CMM
49666,99296442,1,1,2,2,361655,A04P,NaN,RRJ


In [6]:
# Drop ADA/JUDGE from Chrg table
df_dict['Chrg'].drop(columns=['ADA_CODE', 'JUDGE_CODE'], inplace=True)
df_dict['Chrg']

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,BOFI_NBR,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG
0,84000020,1,1,1,255544,19840507,Y,NaN,1,2000.0,2000.0,Y
1,84000133,1,1,1,257683,19840615,Y,NaN,1,500.0,500.0,Y
2,84000154,1,1,1,255696,19840516,Y,NaN,1,0.0,0.0,N
3,84000154,1,1,2,255696,19840516,N,NaN,2,0.0,0.0,N
4,84000180,1,1,1,251021,19840524,Y,NaN,1,0.0,0.0,N
...,...,...,...,...,...,...,...,...,...,...,...,...
591959,99309155,1,1,2,J117134,19991102,N,NaN,1,0.0,0.0,N
591960,99309156,1,1,1,J117135,19991102,N,NaN,1,0.0,0.0,N
591961,99309156,1,1,2,J117135,19991102,Y,NaN,1,0.0,0.0,N
591962,99309155,1,1,3,J117134,19991103,Y,NaN,1,0.0,0.0,N


In [7]:
# Drop ADA/JUDGE from Chrg table
df_dict['Disp'].drop(columns=['DISP_DATE_DISP', 'DISP_CODE_DISP'], inplace=True)
df_dict['Disp']

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,BOFI_NBR,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP
0,0,1,1,1,1,J87355,14:81,NaN,NaN,NaN
1,84000280,1,1,1,1,246712,40:967 (C) (2),ERB,NaN,NaN
2,84000405,1,1,1,1,197485,14:82,NaN,NaN,NaN
3,84000429,1,1,1,1,142503,40:966 (D) (2),DFM,NaN,NaN
4,84000873,2,1,1,1,238381,14:89,CJW,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
574564,99309156,1,1,1,1,J117135,14:69 (B),NaN,NaN,NaN
574565,99309156,1,1,2,1,J117135,14:67.10 (B),NaN,NaN,NaN
574566,99309155,1,1,1,1,J117134,14:69 (B),NaN,NaN,NaN
574567,99309155,1,1,2,1,J117134,14:67.10 (B),NaN,NaN,NaN


In [8]:
# Drop duplicated bofi_nbr from Dfdn
bofi_drop = df_dict['Dfdn'][df_dict['Dfdn'].duplicated(['BOFI_NBR'], keep='first')]['BOFI_NBR']
df_dict['Dfdn'].drop(bofi_drop.index, inplace=True)
# Drop unnecessary columns 
df_dict['Dfdn'].drop(columns=['CCN_NBR', 'ZIP_DFDN'], inplace=True)

In [9]:
# handle juvenile_flag namings
df_dict['Dfdn'].rename(columns={'JUVENILE_FLAG': 'JUVENILE_FLAG_DFDN'}, inplace=True)
df_dict['Caze'].rename(columns={'JUVENILE_FLAG': 'JUVENILE_INVOLVED_FLAG'}, inplace=True)

### Start Merging

In [10]:
# Merge SENT and DFDN
data_merged = pd.merge(df_dict['Sent'], df_dict['Dfdn'], on='BOFI_NBR', how='inner')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_DFDN,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN
1,84000405,1,1,1,1,1,197485,0.640277778,19911125,50,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN
3,84000873,2,1,1,1,1,238381,0.645138889,19860602,50,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN
4,84000873,2,1,2,1,1,238381,23:49.1,19860730,30,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130728,99308206,1,1,4,1,1,J116921,56:21.3,19991104,50,JEAG,N,1,0,0,CNR,N,3,0,0,30.0,0.0,0.0,0.0,0,N,N,N,N,DOC,Y,S,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK
130729,99308206,1,1,5,1,1,J116921,40:966 (D) (1),19991104,50,JEAG,N,0,6,0,CNR,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,DOC,N,NaN,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK
130730,99306774,1,1,3,1,1,J96877,14:67.10 (C),19991115,50,JFAH,N,0,0,7,SIN,N,0,6,0,0.0,0.0,0.0,0.0,0,N,N,N,N,DOC,N,S,B6230,NaN,Y,Y,NaN,19841227,5,5,105,M,N,MBR,BRO,BLK
130731,99308953,1,1,2,1,1,J115276,15:08.4,19991115,50,JFAH,N,0,6,0,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,DOC,N,U,P2000,NaN,Y,Y,NaN,19841111,5,2,115,F,N,DBR,BRO,BLK


In [11]:
data_merged = pd.merge(data_merged, df_dict['Caze'], on='SYS_NBR', how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_DFDN,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_INVOLVED_FLAG,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN
1,84000405,1,1,1,1,1,197485,0.640277778,19911125,50,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN
3,84000873,2,1,1,1,1,238381,0.645138889,19860602,50,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN
4,84000873,2,1,2,1,1,238381,23:49.1,19860730,30,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130728,99308206,1,1,4,1,1,J116921,56:21.3,19991104,50,JEAG,N,1,0,0,CNR,N,3,0,0,30.0,0.0,0.0,0.0,0,N,N,N,N,DOC,Y,S,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN
130729,99308206,1,1,5,1,1,J116921,40:966 (D) (1),19991104,50,JEAG,N,0,6,0,CNR,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,DOC,N,NaN,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN
130730,99306774,1,1,3,1,1,J96877,14:67.10 (C),19991115,50,JFAH,N,0,0,7,SIN,N,0,6,0,0.0,0.0,0.0,0.0,0,N,N,N,N,DOC,N,S,B6230,NaN,Y,Y,NaN,19841227,5,5,105,M,N,MBR,BRO,BLK,JGN,NaN,4.0,1,9928706Q,N,14:67.10 (C),NaN,NaN
130731,99308953,1,1,2,1,1,J115276,15:08.4,19991115,50,JFAH,N,0,6,0,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,DOC,N,U,P2000,NaN,Y,Y,NaN,19841111,5,2,115,F,N,DBR,BRO,BLK,JGN,NaN,3.0,1,9931426Q,N,14:68.4,NaN,NaN


In [12]:
data_merged = pd.merge(data_merged, df_dict['Chrg'], on=['SYS_NBR', 'DFDN_SEQ_NBR', 'AREG_SEQ_NBR', 'CHRG_SEQ_NBR', 'BOFI_NBR'], how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_DFDN,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_INVOLVED_FLAG,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,0.0,0.0,N
1,84000405,1,1,1,1,1,197485,0.640277778,19911125,50,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,5000.0,5000.0,Y
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,2000.0,2000.0,Y
3,84000873,2,1,1,1,1,238381,0.645138889,19860602,50,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,100.0,100.0,Y
4,84000873,2,1,2,1,1,238381,23:49.1,19860730,30,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19860602.0,N,CJW,1.0,0.0,0.0,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130728,99308206,1,1,4,1,1,J116921,56:21.3,19991104,50,JEAG,N,1,0,0,CNR,N,3,0,0,30.0,0.0,0.0,0.0,0,N,N,N,N,DOC,Y,S,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN,19991028.0,N,NaN,1.0,0.0,0.0,N
130729,99308206,1,1,5,1,1,J116921,40:966 (D) (1),19991104,50,JEAG,N,0,6,0,CNR,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,DOC,N,NaN,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN,19991028.0,Y,NaN,1.0,0.0,0.0,N
130730,99306774,1,1,3,1,1,J96877,14:67.10 (C),19991115,50,JFAH,N,0,0,7,SIN,N,0,6,0,0.0,0.0,0.0,0.0,0,N,N,N,N,DOC,N,S,B6230,NaN,Y,Y,NaN,19841227,5,5,105,M,N,MBR,BRO,BLK,JGN,NaN,4.0,1,9928706Q,N,14:67.10 (C),NaN,NaN,19990506.0,Y,NaN,1.0,0.0,0.0,N
130731,99308953,1,1,2,1,1,J115276,15:08.4,19991115,50,JFAH,N,0,6,0,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,DOC,N,U,P2000,NaN,Y,Y,NaN,19841111,5,2,115,F,N,DBR,BRO,BLK,JGN,NaN,3.0,1,9931426Q,N,14:68.4,NaN,NaN,19991109.0,Y,NaN,1.0,0.0,0.0,N


In [13]:
data_merged = pd.merge(data_merged, df_dict['Cgcd'], left_on='CHARGE_CODE_SENT', right_on='CHARGE_CODE', how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_DFDN,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_INVOLVED_FLAG,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CODE,CHARGE_CAT,CHARGE_CLASS
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,0.0,0.0,N,40:967 (C) (2),NAR,3
1,84000405,1,1,1,1,1,197485,0.640277778,19911125,50,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,5000.0,5000.0,Y,NaN,NaN,NaN
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,2000.0,2000.0,Y,40:966 (D) (2),NAR,3
3,84000873,2,1,1,1,1,238381,0.645138889,19860602,50,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,100.0,100.0,Y,NaN,NaN,NaN
4,84000873,2,1,2,1,1,238381,23:49.1,19860730,30,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19860602.0,N,CJW,1.0,0.0,0.0,N,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130728,99308206,1,1,4,1,1,J116921,56:21.3,19991104,50,JEAG,N,1,0,0,CNR,N,3,0,0,30.0,0.0,0.0,0.0,0,N,N,N,N,DOC,Y,S,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN,19991028.0,N,NaN,1.0,0.0,0.0,N,NaN,NaN,NaN
130729,99308206,1,1,5,1,1,J116921,40:966 (D) (1),19991104,50,JEAG,N,0,6,0,CNR,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,DOC,N,NaN,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN,19991028.0,Y,NaN,1.0,0.0,0.0,N,40:966 (D) (1),NAR,4
130730,99306774,1,1,3,1,1,J96877,14:67.10 (C),19991115,50,JFAH,N,0,0,7,SIN,N,0,6,0,0.0,0.0,0.0,0.0,0,N,N,N,N,DOC,N,S,B6230,NaN,Y,Y,NaN,19841227,5,5,105,M,N,MBR,BRO,BLK,JGN,NaN,4.0,1,9928706Q,N,14:67.10 (C),NaN,NaN,19990506.0,Y,NaN,1.0,0.0,0.0,N,14:67.10 (C),NaN,4
130731,99308953,1,1,2,1,1,J115276,15:08.4,19991115,50,JFAH,N,0,6,0,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,DOC,N,U,P2000,NaN,Y,Y,NaN,19841111,5,2,115,F,N,DBR,BRO,BLK,JGN,NaN,3.0,1,9931426Q,N,14:68.4,NaN,NaN,19991109.0,Y,NaN,1.0,0.0,0.0,N,NaN,NaN,NaN


In [14]:
data_merged = pd.merge(data_merged, df_dict['Disp'], on=['SYS_NBR', 'DFDN_SEQ_NBR', 'AREG_SEQ_NBR', 'CHRG_SEQ_NBR',
                                                         'DISP_SEQ_NBR', 'BOFI_NBR'], how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_DFDN,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_INVOLVED_FLAG,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CODE,CHARGE_CAT,CHARGE_CLASS,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),ERB,NaN,NaN
1,84000405,1,1,1,1,1,197485,0.640277778,19911125,50,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,5000.0,5000.0,Y,NaN,NaN,NaN,14:82,NaN,NaN,NaN
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,2000.0,2000.0,Y,40:966 (D) (2),NAR,3,40:966 (D) (2),DFM,NaN,NaN
3,84000873,2,1,1,1,1,238381,0.645138889,19860602,50,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,100.0,100.0,Y,NaN,NaN,NaN,14:89,CJW,NaN,NaN
4,84000873,2,1,2,1,1,238381,23:49.1,19860730,30,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19860602.0,N,CJW,1.0,0.0,0.0,N,NaN,NaN,NaN,15:529.1,CJW,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130728,99308206,1,1,4,1,1,J116921,56:21.3,19991104,50,JEAG,N,1,0,0,CNR,N,3,0,0,30.0,0.0,0.0,0.0,0,N,N,N,N,DOC,Y,S,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN,19991028.0,N,NaN,1.0,0.0,0.0,N,NaN,NaN,NaN,40:981.3,NaN,JEAG,NaN
130729,99308206,1,1,5,1,1,J116921,40:966 (D) (1),19991104,50,JEAG,N,0,6,0,CNR,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,DOC,N,NaN,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN,19991028.0,Y,NaN,1.0,0.0,0.0,N,40:966 (D) (1),NAR,4,40:966 (D) (1),NaN,JEAG,NaN
130730,99306774,1,1,3,1,1,J96877,14:67.10 (C),19991115,50,JFAH,N,0,0,7,SIN,N,0,6,0,0.0,0.0,0.0,0.0,0,N,N,N,N,DOC,N,S,B6230,NaN,Y,Y,NaN,19841227,5,5,105,M,N,MBR,BRO,BLK,JGN,NaN,4.0,1,9928706Q,N,14:67.10 (C),NaN,NaN,19990506.0,Y,NaN,1.0,0.0,0.0,N,14:67.10 (C),NaN,4,14:67.10 (C),NaN,JFAH,NaN
130731,99308953,1,1,2,1,1,J115276,15:08.4,19991115,50,JFAH,N,0,6,0,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,DOC,N,U,P2000,NaN,Y,Y,NaN,19841111,5,2,115,F,N,DBR,BRO,BLK,JGN,NaN,3.0,1,9931426Q,N,14:68.4,NaN,NaN,19991109.0,Y,NaN,1.0,0.0,0.0,N,NaN,NaN,NaN,14:68.4,NaN,JFAH,NaN


In [15]:
# One sentencing can have multiple conditions -> thus the number or rows increased
data_merged = pd.merge(data_merged, df_dict['Prcn'], on=['SYS_NBR', 'DFDN_SEQ_NBR', 'AREG_SEQ_NBR', 'CHRG_SEQ_NBR', 
                                                         'DISP_SEQ_NBR', 'SENT_SEQ_NBR'], how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_DFDN,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_INVOLVED_FLAG,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CODE,CHARGE_CAT,CHARGE_CLASS,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP,PRCN_SEQ_NBR,CONDITION_CODE,CONDITION_YEARS,CONDITION_MONTHS,CONDITION_DAYS,CONDITION_HOURS
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),ERB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84000405,1,1,1,1,1,197485,0.640277778,19911125,50,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,5000.0,5000.0,Y,NaN,NaN,NaN,14:82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,2000.0,2000.0,Y,40:966 (D) (2),NAR,3,40:966 (D) (2),DFM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,84000873,2,1,1,1,1,238381,0.645138889,19860602,50,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,100.0,100.0,Y,NaN,NaN,NaN,14:89,CJW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,84000873,2,1,2,1,1,238381,23:49.1,19860730,30,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19860602.0,N,CJW,1.0,0.0,0.0,N,NaN,NaN,NaN,15:529.1,CJW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132407,99308206,1,1,4,1,1,J116921,56:21.3,19991104,50,JEAG,N,1,0,0,CNR,N,3,0,0,30.0,0.0,0.0,0.0,0,N,N,N,N,DOC,Y,S,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN,19991028.0,N,NaN,1.0,0.0,0.0,N,NaN,NaN,NaN,40:981.3,NaN,JEAG,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132408,99308206,1,1,5,1,1,J116921,40:966 (D) (1),19991104,50,JEAG,N,0,6,0,CNR,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,DOC,N,NaN,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN,19991028.0,Y,NaN,1.0,0.0,0.0,N,40:966 (D) (1),NAR,4,40:966 (D) (1),NaN,JEAG,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132409,99306774,1,1,3,1,1,J96877,14:67.10 (C),19991115,50,JFAH,N,0,0,7,SIN,N,0,6,0,0.0,0.0,0.0,0.0,0,N,N,N,N,DOC,N,S,B6230,NaN,Y,Y,NaN,19841227,5,5,105,M,N,MBR,BRO,BLK,JGN,NaN,4.0,1,9928706Q,N,14:67.10 (C),NaN,NaN,19990506.0,Y,NaN,1.0,0.0,0.0,N,14:67.10 (C),NaN,4,14:67.10 (C),NaN,JFAH,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132410,99308953,1,1,2,1,1,J115276,15:08.4,19991115,50,JFAH,N,0,6,0,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,DOC,N,

In [16]:
data_merged = pd.merge(data_merged, df_dict['Bond'], on=['SYS_NBR', 'DFDN_SEQ_NBR', 'AREG_SEQ_NBR', 'CHRG_SEQ_NBR',
                                                         'BOFI_NBR', 'SUB_CHARGE_CODE'], how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_DFDN,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_INVOLVED_FLAG,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CODE,CHARGE_CAT,CHARGE_CLASS,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP,PRCN_SEQ_NBR,CONDITION_CODE,CONDITION_YEARS,CONDITION_MONTHS,CONDITION_DAYS,CONDITION_HOURS,BOND_SEQ_NBR,BOND_STATUS_CODE,BOND_TYPE_CODE
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),ERB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84000405,1,1,1,1,1,197485,0.640277778,19911125,50,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,5000.0,5000.0,Y,NaN,NaN,NaN,14:82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CMM
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,2000.0,2000.0,Y,40:966 (D) (2),NAR,3,40:966 (D) (2),DFM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,CMM
3,84000873,2,1,1,1,1,238381,0.645138889,19860602,50,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,100.0,100.0,Y,NaN,NaN,NaN,14:89,CJW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CBP
4,84000873,2,1,2,1,1,238381,23:49.1,19860730,30,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19860602.0,N,CJW,1.0,0.0,0.0,N,NaN,NaN,NaN,15:529.1,CJW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132407,99308206,1,1,4,1,1,J116921,56:21.3,19991104,50,JEAG,N,1,0,0,CNR,N,3,0,0,30.0,0.0,0.0,0.0,0,N,N,N,N,DOC,Y,S,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN,19991028.0,N,NaN,1.0,0.0,0.0,N,NaN,NaN,NaN,40:981.3,NaN,JEAG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132408,99308206,1,1,5,1,1,J116921,40:966 (D) (1),19991104,50,JEAG,N,0,6,0,CNR,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,DOC,N,NaN,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN,19991028.0,Y,NaN,1.0,0.0,0.0,N,40:966 (D) (1),NAR,4,40:966 (D) (1),NaN,JEAG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132409,99306774,1,1,3,1,1,J96877,14:67.10 (C),19991115,50,JFAH,N,0,0,7,SIN,N,0,6,0,0.0,0.0,0.0,0.0,0,N,N,N,N,DOC,N,S,B6230,NaN,Y,Y,NaN,19841227,5,5,105,M,N,MBR,BRO,BLK,JGN,NaN,4.0,1,9928706Q,N,14:67.10 (C),NaN,NaN,19990506.0,Y,NaN,1.0,0.0,0.0,N,14:67.10 (C),NaN,4,14:67.10 (C),NaN,JFAH,

In [17]:
data_merged = pd.merge(data_merged, df_dict['Judge'], left_on='JUDGE_CODE_SENT', right_on='JUDGE_CODE', how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_DFDN,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_INVOLVED_FLAG,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CODE,CHARGE_CAT,CHARGE_CLASS,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP,PRCN_SEQ_NBR,CONDITION_CODE,CONDITION_YEARS,CONDITION_MONTHS,CONDITION_DAYS,CONDITION_HOURS,BOND_SEQ_NBR,BOND_STATUS_CODE,BOND_TYPE_CODE,JUDGE_CODE,SEX_JUDGE,RACE_JUDGE,AGE_JUDGE,PARTY_JUDGE
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),ERB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84000405,1,1,1,1,1,197485,0.640277778,19911125,50,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,5000.0,5000.0,Y,NaN,NaN,NaN,14:82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CMM,NaN,NaN,NaN,NaN,NaN
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,2000.0,2000.0,Y,40:966 (D) (2),NAR,3,40:966 (D) (2),DFM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,CMM,NaN,NaN,NaN,NaN,NaN
3,84000873,2,1,1,1,1,238381,0.645138889,19860602,50,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,100.0,100.0,Y,NaN,NaN,NaN,14:89,CJW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CBP,NaN,NaN,NaN,NaN,NaN
4,84000873,2,1,2,1,1,238381,23:49.1,19860730,30,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19860602.0,N,CJW,1.0,0.0,0.0,N,NaN,NaN,NaN,15:529.1,CJW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132407,99308206,1,1,4,1,1,J116921,56:21.3,19991104,50,JEAG,N,1,0,0,CNR,N,3,0,0,30.0,0.0,0.0,0.0,0,N,N,N,N,DOC,Y,S,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN,19991028.0,N,NaN,1.0,0.0,0.0,N,NaN,NaN,NaN,40:981.3,NaN,JEAG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JEAG,F,W,64.0,D
132408,99308206,1,1,5,1,1,J116921,40:966 (D) (1),19991104,50,JEAG,N,0,6,0,CNR,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,DOC,N,NaN,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN,19991028.0,Y,NaN,1.0,0.0,0.0,N,40:966 (D) (1),NAR,4,40:966 (D) (1),NaN,JEAG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JEAG,F,W,64.0,D
132409,99306774,1,1,3,1,1,J96877,14:67.10 (C),19991115,50,JFAH,N,0,0,7,SIN,N,0,6

In [18]:
df_dict['Dsum'] = df_dict['Dsum'][df_dict['Dsum']['SCREEN_ADA_CODE'].notna() | df_dict['Dsum']['TRIAL_ADA_CODE'].notna() \
                                  | df_dict['Dsum']['TRIAL_DISP_CODE'].notna()]
df_dict['Dsum'] = df_dict['Dsum'].drop_duplicates()
data_merged = pd.merge(data_merged, df_dict['Dsum'], on=['SYS_NBR', 'BOFI_NBR'], how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_DFDN,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_INVOLVED_FLAG,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CODE,CHARGE_CAT,CHARGE_CLASS,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP,PRCN_SEQ_NBR,CONDITION_CODE,CONDITION_YEARS,CONDITION_MONTHS,CONDITION_DAYS,CONDITION_HOURS,BOND_SEQ_NBR,BOND_STATUS_CODE,BOND_TYPE_CODE,JUDGE_CODE,SEX_JUDGE,RACE_JUDGE,AGE_JUDGE,PARTY_JUDGE,SCREEN_ADA_CODE,TRIAL_ADA_CODE,TRIAL_DISP_CODE
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),ERB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84000405,1,1,1,1,1,197485,0.640277778,19911125,50,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,5000.0,5000.0,Y,NaN,NaN,NaN,14:82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CMM,NaN,NaN,NaN,NaN,NaN,NaN,ROBA,50.0
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,2000.0,2000.0,Y,40:966 (D) (2),NAR,3,40:966 (D) (2),DFM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,CMM,NaN,NaN,NaN,NaN,NaN,NaN,CBEL,50.0
3,84000873,2,1,1,1,1,238381,0.645138889,19860602,50,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,100.0,100.0,Y,NaN,NaN,NaN,14:89,CJW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CBP,NaN,NaN,NaN,NaN,NaN,NaN,RMOS,50.0
4,84000873,2,1,2,1,1,238381,23:49.1,19860730,30,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19860602.0,N,CJW,1.0,0.0,0.0,N,NaN,NaN,NaN,15:529.1,CJW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RMOS,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132412,99308206,1,1,4,1,1,J116921,56:21.3,19991104,50,JEAG,N,1,0,0,CNR,N,3,0,0,30.0,0.0,0.0,0.0,0,N,N,N,N,DOC,Y,S,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN,19991028.0,N,NaN,1.0,0.0,0.0,N,NaN,NaN,NaN,40:981.3,NaN,JEAG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JEAG,F,W,64.0,D,ADAI,BHIL,50.0
132413,99308206,1,1,5,1,1,J116921,40:966 (D) (1),19991104,50,JEAG,N,0,6,0,CNR,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,DOC,N,NaN,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN,19991028.0,Y,NaN,1.0,0.0,0.0,N,40:966 (D) (1),NAR,4,40:966 (D) (1),NaN

In [19]:
df_dict['Screen_Ada'] = df_dict['Ada']
df_dict['Screen_Ada'].columns = ['SCREEN_ADA_CODE', 'ADA_NAME_SCREEN_ADA', 'DOB_SCREEN_ADA', \
                                 'RACE_SCREEN_ADA', 'SEX_SCREEN_ADA', 'PARTY_SCREEN_ADA']
data_merged = pd.merge(data_merged, df_dict['Screen_Ada'], on='SCREEN_ADA_CODE', how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_DFDN,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_INVOLVED_FLAG,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CODE,CHARGE_CAT,CHARGE_CLASS,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP,PRCN_SEQ_NBR,CONDITION_CODE,CONDITION_YEARS,CONDITION_MONTHS,CONDITION_DAYS,CONDITION_HOURS,BOND_SEQ_NBR,BOND_STATUS_CODE,BOND_TYPE_CODE,JUDGE_CODE,SEX_JUDGE,RACE_JUDGE,AGE_JUDGE,PARTY_JUDGE,SCREEN_ADA_CODE,TRIAL_ADA_CODE,TRIAL_DISP_CODE,ADA_NAME_SCREEN_ADA,DOB_SCREEN_ADA,RACE_SCREEN_ADA,SEX_SCREEN_ADA,PARTY_SCREEN_ADA
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),ERB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84000405,1,1,1,1,1,197485,0.640277778,19911125,50,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,5000.0,5000.0,Y,NaN,NaN,NaN,14:82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CMM,NaN,NaN,NaN,NaN,NaN,NaN,ROBA,50.0,NaN,NaN,NaN,NaN,NaN
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,2000.0,2000.0,Y,40:966 (D) (2),NAR,3,40:966 (D) (2),DFM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,CMM,NaN,NaN,NaN,NaN,NaN,NaN,CBEL,50.0,NaN,NaN,NaN,NaN,NaN
3,84000873,2,1,1,1,1,238381,0.645138889,19860602,50,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,100.0,100.0,Y,NaN,NaN,NaN,14:89,CJW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CBP,NaN,NaN,NaN,NaN,NaN,NaN,RMOS,50.0,NaN,NaN,NaN,NaN,NaN
4,84000873,2,1,2,1,1,238381,23:49.1,19860730,30,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19860602.0,N,CJW,1.0,0.0,0.0,N,NaN,NaN,NaN,15:529.1,CJW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RMOS,50.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132412,99308206,1,1,4,1,1,J116921,56:21.3,19991104,50,JEAG,N,1,0,0,CNR,N,3,0,0,30.0,0.0,0.0,0.0,0,N,N,N,N,DOC,Y,S,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,N,MBR,BRO,BLK,JNP,NaN,4.0,1,9930205Q,N,40:966 (D) (1),NaN,NaN,19991028.0,N,NaN,1.0,0.0,0.0,N,NaN,NaN,NaN,40:981.3,NaN,JEAG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JEAG,F,W,64.0,D,ADAI,BHIL,50.0,"DAIGLE, AMIE (STASSI)",19710916.0,W,F,R
132413,99308206,1,1,5,1,1,J116921,40:966 (D) (1

In [20]:
df_dict['Trial_Ada'] = df_dict['Ada']
df_dict['Trial_Ada'].columns = ['TRIAL_ADA_CODE', 'ADA_NAME_TRIAL_ADA', 'DOB_TRIAL_ADA', \
                                'RACE_TRIAL_ADA', 'SEX_TRIAL_ADA', 'PARTY_TRIAL_ADA']
data_merged = pd.merge(data_merged, df_dict['Trial_Ada'], on='TRIAL_ADA_CODE', how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_DFDN,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_INVOLVED_FLAG,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CODE,CHARGE_CAT,CHARGE_CLASS,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP,PRCN_SEQ_NBR,CONDITION_CODE,CONDITION_YEARS,CONDITION_MONTHS,CONDITION_DAYS,CONDITION_HOURS,BOND_SEQ_NBR,BOND_STATUS_CODE,BOND_TYPE_CODE,JUDGE_CODE,SEX_JUDGE,RACE_JUDGE,AGE_JUDGE,PARTY_JUDGE,SCREEN_ADA_CODE,TRIAL_ADA_CODE,TRIAL_DISP_CODE,ADA_NAME_SCREEN_ADA,DOB_SCREEN_ADA,RACE_SCREEN_ADA,SEX_SCREEN_ADA,PARTY_SCREEN_ADA,ADA_NAME_TRIAL_ADA,DOB_TRIAL_ADA,RACE_TRIAL_ADA,SEX_TRIAL_ADA,PARTY_TRIAL_ADA
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),ERB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84000405,1,1,1,1,1,197485,0.640277778,19911125,50,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,5000.0,5000.0,Y,NaN,NaN,NaN,14:82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CMM,NaN,NaN,NaN,NaN,NaN,NaN,ROBA,50.0,NaN,NaN,NaN,NaN,NaN,"OBANNON, ROBIN CATHERINE",19630000.0,T,F,R
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,2000.0,2000.0,Y,40:966 (D) (2),NAR,3,40:966 (D) (2),DFM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,CMM,NaN,NaN,NaN,NaN,NaN,NaN,CBEL,50.0,NaN,NaN,NaN,NaN,NaN,"BELEW, CHRISTINA A H",19570409.0,NaN,NaN,NaN
3,84000873,2,1,1,1,1,238381,0.645138889,19860602,50,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,100.0,100.0,Y,NaN,NaN,NaN,14:89,CJW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CBP,NaN,NaN,NaN,NaN,NaN,NaN,RMOS,50.0,NaN,NaN,NaN,NaN,NaN,"MOSELEY, ROCKNE L",19520000.0,W,M,D
4,84000873,2,1,2,1,1,238381,23:49.1,19860730,30,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19860602.0,N,CJW,1.0,0.0,0.0,N,NaN,NaN,NaN,15:529.1,CJW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RMOS,50.0,NaN,NaN,NaN,NaN,NaN,"MOSELEY, ROCKNE L",19520000.0,W,M,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132412,99308206,1,1,4,1,1,J116921,56:21.3,19991104,50,JEAG,N,1,0,0,CNR,N,3,0,0,30.0,0.0,0.0,0.0,0,N,N,N,N,DOC,Y,S,R1525,NaN,Y,Y,NaN,19860803,5,2,102,M,

In [21]:
data_merged = pd.merge(data_merged, df_dict['Event'], on=['SYS_NBR', 'DFDN_SEQ_NBR', 'AREG_SEQ_NBR', 'CHRG_SEQ_NBR', 
                                                          'DISP_SEQ_NBR', 'SENT_SEQ_NBR', 'BOND_SEQ_NBR', 'BOFI_NBR'], how='left')
data_merged.drop(columns=['ADA_CODE_EVENT'], inplace=True)
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_DFDN,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_INVOLVED_FLAG,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CODE,CHARGE_CAT,CHARGE_CLASS,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP,PRCN_SEQ_NBR,CONDITION_CODE,CONDITION_YEARS,CONDITION_MONTHS,CONDITION_DAYS,CONDITION_HOURS,BOND_SEQ_NBR,BOND_STATUS_CODE,BOND_TYPE_CODE,JUDGE_CODE,SEX_JUDGE,RACE_JUDGE,AGE_JUDGE,PARTY_JUDGE,SCREEN_ADA_CODE,TRIAL_ADA_CODE,TRIAL_DISP_CODE,ADA_NAME_SCREEN_ADA,DOB_SCREEN_ADA,RACE_SCREEN_ADA,SEX_SCREEN_ADA,PARTY_SCREEN_ADA,ADA_NAME_TRIAL_ADA,DOB_TRIAL_ADA,RACE_TRIAL_ADA,SEX_TRIAL_ADA,PARTY_TRIAL_ADA,EVENT_CODE_EVENT
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),ERB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84000405,1,1,1,1,1,197485,0.640277778,19911125,50,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,5000.0,5000.0,Y,NaN,NaN,NaN,14:82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CMM,NaN,NaN,NaN,NaN,NaN,NaN,ROBA,50.0,NaN,NaN,NaN,NaN,NaN,"OBANNON, ROBIN CATHERINE",19630000.0,T,F,R,NaN
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,2000.0,2000.0,Y,40:966 (D) (2),NAR,3,40:966 (D) (2),DFM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,CMM,NaN,NaN,NaN,NaN,NaN,NaN,CBEL,50.0,NaN,NaN,NaN,NaN,NaN,"BELEW, CHRISTINA A H",19570409.0,NaN,NaN,NaN,NaN
3,84000873,2,1,1,1,1,238381,0.645138889,19860602,50,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,100.0,100.0,Y,NaN,NaN,NaN,14:89,CJW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CBP,NaN,NaN,NaN,NaN,NaN,NaN,RMOS,50.0,NaN,NaN,NaN,NaN,NaN,"MOSELEY, ROCKNE L",19520000.0,W,M,D,NaN
4,84000873,2,1,2,1,1,238381,23:49.1,19860730,30,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19860602.0,N,CJW,1.0,0.0,0.0,N,NaN,NaN,NaN,15:529.1,CJW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RMOS,50.0,NaN,NaN,NaN,NaN,NaN,"MOSELEY, ROCKNE L",19520000.0,W,M,D,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132412,99308206,1,1,4,1,1,J116921,56:21.3,19991104,50,JEAG,N,1,0,0,CNR,N,3,0,0,30.0,0.0,0.0,0.0,0,N,N,N,N,DOC,

In [22]:
data_merged = pd.merge(data_merged, df_dict['Parm'], left_on='EVENT_CODE_EVENT', right_on='EVENT_CODE_PARM', how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_DFDN,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_INVOLVED_FLAG,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CODE,CHARGE_CAT,CHARGE_CLASS,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP,PRCN_SEQ_NBR,CONDITION_CODE,CONDITION_YEARS,CONDITION_MONTHS,CONDITION_DAYS,CONDITION_HOURS,BOND_SEQ_NBR,BOND_STATUS_CODE,BOND_TYPE_CODE,JUDGE_CODE,SEX_JUDGE,RACE_JUDGE,AGE_JUDGE,PARTY_JUDGE,SCREEN_ADA_CODE,TRIAL_ADA_CODE,TRIAL_DISP_CODE,ADA_NAME_SCREEN_ADA,DOB_SCREEN_ADA,RACE_SCREEN_ADA,SEX_SCREEN_ADA,PARTY_SCREEN_ADA,ADA_NAME_TRIAL_ADA,DOB_TRIAL_ADA,RACE_TRIAL_ADA,SEX_TRIAL_ADA,PARTY_TRIAL_ADA,EVENT_CODE_EVENT,EVENT_CODE_PARM,BOND_EVENT_FLAG,WHO_CODE
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),ERB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84000405,1,1,1,1,1,197485,0.640277778,19911125,50,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,5000.0,5000.0,Y,NaN,NaN,NaN,14:82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CMM,NaN,NaN,NaN,NaN,NaN,NaN,ROBA,50.0,NaN,NaN,NaN,NaN,NaN,"OBANNON, ROBIN CATHERINE",19630000.0,T,F,R,NaN,NaN,NaN,NaN
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,2000.0,2000.0,Y,40:966 (D) (2),NAR,3,40:966 (D) (2),DFM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,CMM,NaN,NaN,NaN,NaN,NaN,NaN,CBEL,50.0,NaN,NaN,NaN,NaN,NaN,"BELEW, CHRISTINA A H",19570409.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,84000873,2,1,1,1,1,238381,0.645138889,19860602,50,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,100.0,100.0,Y,NaN,NaN,NaN,14:89,CJW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CBP,NaN,NaN,NaN,NaN,NaN,NaN,RMOS,50.0,NaN,NaN,NaN,NaN,NaN,"MOSELEY, ROCKNE L",19520000.0,W,M,D,NaN,NaN,NaN,NaN
4,84000873,2,1,2,1,1,238381,23:49.1,19860730,30,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815,0,0,0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19860602.0,N,CJW,1.0,0.0,0.0,N,NaN,NaN,NaN,15:529.1,CJW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RMOS,50.0,NaN,NaN,NaN,NaN,NaN,"MOSELEY, ROCKNE L",19520000.0,W,M,D,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [23]:
for name in data_merged.columns:
    if name.endswith('_x'):
        print(name)

In [25]:
for name in data_merged.columns:
    print(name)

SYS_NBR
DFDN_SEQ_NBR
AREG_SEQ_NBR
CHRG_SEQ_NBR
DISP_SEQ_NBR
SENT_SEQ_NBR
BOFI_NBR
CHARGE_CODE_SENT
DISP_DATE_SENT
DISP_CODE_SENT
JUDGE_CODE_SENT
APPEAL_FLAG
CONFINEMENT_YEARS
CONFINEMENT_MONTHS
CONFINEMENT_DAYS
SENTENCE_TYPE
HABITUAL_OFFENDER_FLAG_SENT
ACTIVE_PROB_YEARS
ACTIVE_PROB_MONTHS
ACTIVE_PROB_DAYS
FINE_AMOUNT
VICTIM_FUND_AMOUNT
RESTITUTION_AMOUNT
INDIGENT_DEFENDER_FUND_AMOUNT
JUDICIAL_FUND_AMOUNT
FINE_SUSPENDED_FLAG
COMMUNITY_SERVICE_FLAG
CONDITION_OF_PROBATION_FLAG
CREDIT_TIME_SERVED_FLAG
SENTENCE_LOCATION
SUSPENDED_SENTENCE_FLAG
PROBATION_TYPE
PHONIC_KEY
CUSTODY_CODE
JUVENILE_FLAG_DFDN
CRIMINAL_FLAG
FBI_NBR
DOB_DFDN
HEIGHT_FEET
HEIGHT_INCHES
WEIGHT
SEX_DFDN
RACE_DFDN
SKIN_COLOR
EYE_COLOR
HAIR_COLOR
CASE_TYPE
CASE_TYPE_2
CASE_CLASS
NBR_OF_DFDN
CASE_NBR
JUVENILE_INVOLVED_FLAG
LEAD_CHARGE_CODE
CASE_CAT_1
CASE_CAT_2
CHARGE_DATE
LEAD_CHARGE_FLAG
SUB_CHARGE_CODE
COUNTS
BOND_MADE_AMOUNT
BOND_SET_AMOUNT
BOND_MADE_FLAG
CHARGE_CODE
CHARGE_CAT
CHARGE_CLASS
CHARGE_CODE_DISP
SUB_CHARGE_CO

In [27]:
temp2 = data_merged.groupby(['BOFI_NBR', 'CHARGE_CODE']).count()

In [28]:
temp2[temp2['SYS_NBR'] > 1]

,,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_DFDN,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_INVOLVED_FLAG,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CAT,CHARGE_CLASS,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP,PRCN_SEQ_NBR,CONDITION_CODE,CONDITION_YEARS,CONDITION_MONTHS,CONDITION_DAYS,CONDITION_HOURS,BOND_SEQ_NBR,BOND_STATUS_CODE,BOND_TYPE_CODE,JUDGE_CODE,SEX_JUDGE,RACE_JUDGE,AGE_JUDGE,PARTY_JUDGE,SCREEN_ADA_CODE,TRIAL_ADA_CODE,TRIAL_DISP_CODE,ADA_NAME_SCREEN_ADA,DOB_SCREEN_ADA,RACE_SCREEN_ADA,SEX_SCREEN_ADA,PARTY_SCREEN_ADA,ADA_NAME_TRIAL_ADA,DOB_TRIAL_ADA,RACE_TRIAL_ADA,SEX_TRIAL_ADA,PARTY_TRIAL_ADA,EVENT_CODE_EVENT,EVENT_CODE_PARM,BOND_EVENT_FLAG,WHO_CODE
BOFI_NBR,CHARGE_CODE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100565,40:967 (B) (1),2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,0,2,2,2,2,2,2,2,2,2,0,0,0,2,0,2,2,2,2,2,0,0,2,2,2,2,2,2,2,2,2,2,2,0,0,0,0,0,0,0,0,2,0,2,0,0,0,0,0,0,2,2,0,0,0,0,0,2,2,2,2,2,0,0,0,0
101271,40:967 (C) (2),3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,0,3,3,3,3,3,3,3,3,3,3,3,3,3,0,3,3,3,3,3,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0
101365,40:966 (A) (1),3,3,3,3,3,3,3,3,3,1,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,1,3,0,3,0,3,3,3,3,3,3,3,3,3,3,3,3,3,0,3,3,3,3,3,0,0,3,3,3,3,3,3,3,3,3,3,3,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,3,3,3,3,3,3,3,3,3,3,3,3,2,0,0,0,0
101547,40:967 (C) (2),3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,1,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,0,3,3,3,3,3,0,0,3,3,3,3,3,3,3,3,3,3,3,3,1,0,0,0,0,0,0,0,0,0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0
101858,40:967 (B) (1),2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,0,0,2,2,2,2,2,2,2,2,2,2,2,2,2,0,0,0,0,0,0,0,0,0,2,2,2,2,2,2,2,2,2,2,0,2,0,2,2,2,2,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Y101448,14:34.3,2,2,2,2,2,2,2,2,2,0,2,2,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2,2,2,0,2,2,0,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,0,0,2,2,0,2,2,2,2,0,2,2,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,2,2,2,2,2,2,0,0,0,0,0,0,0,0,0
Y101600,14:56 (B) (1),3,3,3,3,3,3,3,3,3,3,3,3,3,3,0,3,3,3,3,3,3,3,3,3,3,3,3,3,0,3,3,3,0,3,3,0,3,3,3,3,3,3,3,3,3,3,0,3,3,3,3,3,0,0,3,3,0,3,3,3,3,0,3,3,0,3,0,0,0,0,0,0,0,0,0,0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0
Y101713,14:35,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,0,2,2,0,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,0,0,2,2,0,2,2,2,2,0,2,2,0,2,0,0,0,0,0,0,0,0,0,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,0,0,0
